In [3]:
%pip install torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import openai
from sentence_transformers import SentenceTransformer, util
import torch

# CSV 파일에서 커피 데이터 읽어오기
coffee_data = pd.read_csv('starbucks.csv')


In [ ]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# .env 파일에서 OPENAI_SECRET_KEY 환경 변수 읽기 # ChatGPT API 키 설정
openai.api_key = os.getenv("OPENAI_SECRET_KEY")

# ChatGPT로 응답 생성
def generate_response(query):
    response = openai.Completion.create(
        engine="davinci",
        prompt=query,
        max_tokens=100
    )
    return response.choices[0].text.strip()


In [ ]:
def recommend_coffee():
    query = "I would like a coffee recommendation"
    response = generate_response(query)
    return response


In [ ]:
def describe_coffee(coffee_name):
    # 커피 이름을 이용하여 상세 정보를 가져오는 로직 추가
    coffee_info = coffee_data[coffee_data['name'] == coffee_name]
    
    if not coffee_info.empty:
        description = coffee_info['content'].values[0]
        return description
    else:
        return "I'm sorry, I don't have information about that coffee."



In [ ]:
def provide_additional_info(coffee_name):
    # 커피 이름을 이용하여 추가 정보를 가져오는 로직 추가
    coffee_info = coffee_data[coffee_data['name'] == coffee_name]
    
    if not coffee_info.empty:
        additional_info = f"Caffeine: {coffee_info['detail.caffeine'].values[0]} mg\nKcal: {coffee_info['detail.kcal'].values[0]}\nSaturated Fat: {coffee_info['detail.sat_FAT'].values[0]} g\nSodium: {coffee_info['detail.sodium'].values[0]} mg\nSugars: {coffee_info['detail.sugars'].values[0]} g"
        return additional_info
    else:
        return "I'm sorry, I don't have information about that coffee."



In [ ]:
def main(query):
    if "recommend coffee" in query:
        response = recommend_coffee()
    elif "describe" in query:
        coffee_name = "coffee"  # 여기에 커피 이름을 추출하는 로직 추가
        response = describe_coffee(coffee_name)
    elif "additional info" in query:
        coffee_name = "coffee"  # 여기에 커피 이름을 추출하는 로직 추가
        response = provide_additional_info(coffee_name)
    else:
        response = generate_response(query)
    
    return response


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# 커피 이름 추출을 위한 Sentence Transformer 모델 초기화
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 사용자 질문에서 커피 이름 추출
def extract_coffee_name(query):
    # 커피 데이터에서 커피 이름 목록 추출
    coffee_names = coffee_data['name'].tolist()
    
    # 입력 질문을 Sentence Transformer 모델로 임베딩
    query_embedding = model.encode(query, convert_to_tensor=True)

    # 모든 커피 이름에 대한 임베딩 생성
    coffee_name_embeddings = model.encode(coffee_names, convert_to_tensor=True)

    # 사용자 질문과 모든 커피 이름 간의 코사인 유사도 계산
    similarities = util.pytorch_cos_sim(query_embedding, coffee_name_embeddings)[0]

    # 유사도가 가장 높은 커피 이름 추출
    closest_coffee_name = coffee_names[similarities.argmax()]

    return closest_coffee_name

# 커피 이름 추출
coffee_name = extract_coffee_name(query)
